In [1]:

from pathlib import Path
import pandas as pd, itertools
from chan.zhongshu_hierarchy import build_zhongshu_hierarchy, summarize_hierarchy

ROOT = Path.cwd().parents[0]
CHAN = ROOT / "chan_data"

symbols = ["BTCUSDT", "ETHUSDT"]
tfs = ["5m", "15m", "30m", "1h", "4h"]
whiches = ["fast", "slow"]


modes = ["discovery", "clean"]
min_ratio_list = [0.0005, 0.001, 0.002]
strict_alt_list = [False, True]
allow_overlap_list = [True, False]

results = []

for sym, tf, which in itertools.product(symbols, tfs, whiches):
    p = CHAN / f"zhongshu_{which}" / sym / f"{tf}_zhongshu.csv"
    if not p.exists():
        continue
    l1 = pd.read_csv(p, parse_dates=["timestamp_start", "timestamp_end"])
    if l1.empty:
        continue

    for mode, min_ratio, alt, allow in itertools.product(modes, min_ratio_list, strict_alt_list, allow_overlap_list):
        try:
            levels, pmap = build_zhongshu_hierarchy(
                l1, max_levels=3, mode=mode,
                min_ratio=min_ratio, strict_alt=alt,
                allow_overlap_zs=allow, enable_quality=True)
        except Exception as e:
            results.append(dict(symbol=sym, tf=tf, which=which, mode=mode,
                                 min_ratio=min_ratio, strict_alt=alt, allow_overlap=allow,
                                 ok=False, error=str(e)))
            continue

        summary = summarize_hierarchy(levels, pmap)
        n1 = len(levels[0]) if len(levels) > 0 else 0
        n2 = len(levels[1]) if len(levels) > 1 else 0
        n3 = len(levels[2]) if len(levels) > 2 else 0
        results.append(dict(symbol=sym, tf=tf, which=which, mode=mode,
                            min_ratio=min_ratio, strict_alt=alt, allow_overlap=allow,
                            ok=True, n_L1=n1, n_L2=n2, n_L3=n3,
                            width_median_L2=summary["levels"][1].get("width_median") if len(summary["levels"]) > 1 else None,
                            overlap_L2=summary["levels"][1].get("time_overlap") if len(summary["levels"]) > 1 else None))

zs_result = pd.DataFrame(results)
zs_result.sort_values("n_L2", ascending=False)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
zs_result.sort_values(["symbol", "tf", "which", "mode", "min_ratio", "strict_alt"])


def pick_best(df):
    rows = []
    for (sym, tf, which), grp in df.groupby(["symbol", "tf", "which"]):
        g = grp[grp["n_L2"] > 0].copy()
        if g.empty:
            continue
        best = g.sort_values("n_L2", ascending=False).iloc[0]
        rows.append(best)
    return pd.DataFrame(rows)

In [9]:

from pathlib import Path
import pandas as pd, itertools
from chan.zhongshu_hierarchy import build_zhongshu_hierarchy, summarize_hierarchy

ROOT = Path.cwd().parents[0]
CHAN = ROOT / "chan_data"

symbols = ["BTCUSDT", "ETHUSDT"]
tfs = ["5m", "15m", "30m", "1h", "4h"]
whiches = ["fast"]

results = []

for sym, tf, which in itertools.product(symbols, tfs, whiches):
    p = CHAN / f"zhongshu_{which}" / sym / f"{tf}_zhongshu.csv"
    if not p.exists():
        continue
    l1 = pd.read_csv(p, parse_dates=["timestamp_start", "timestamp_end"])
    if l1.empty:
        continue

    try:
        levels, pmap = build_zhongshu_hierarchy(
            l1,
            max_levels=3,
            mode="discovery",
            min_ratio=0.0005,
            strict_alt=False,
            allow_overlap_zs=True,
            enable_quality=True,
        )
    except Exception as e:
        print(f"[error] {sym}-{tf}-{which}: {e}")
        continue

    summary = summarize_hierarchy(levels, pmap)
    n1 = len(levels[0]) if len(levels) > 0 else 0
    n2 = len(levels[1]) if len(levels) > 1 else 0
    n3 = len(levels[2]) if len(levels) > 2 else 0
    results.append(dict(
        symbol=sym, tf=tf, which=which,
        n_L1=n1, n_L2=n2, n_L3=n3,
        width_median_L2=summary["levels"][1].get("width_median") if len(summary["levels"]) > 1 else None,
        overlap_L2=summary["levels"][1].get("time_overlap") if len(summary["levels"]) > 1 else None
    ))

df = pd.DataFrame(results)
df.to_csv(ROOT / "reports" / "zhongshu_hierarchy_clean_fast.csv", index=False)
df


symbol   tf which  n_L1  n_L2  n_L3  width_median_L2  overlap_L2
0  BTCUSDT   5m  fast    18     0     0              NaN         NaN
1  BTCUSDT  15m  fast    22     2     0         0.008273         0.0
2  BTCUSDT  30m  fast    20     1     0         0.011160         0.0
3  BTCUSDT   1h  fast    16     0     0              NaN         NaN
4  BTCUSDT   4h  fast     5     0     0              NaN         NaN
5  ETHUSDT   5m  fast    73     3     0         0.005165         0.0
6  ETHUSDT  15m  fast    50     3     0         0.003961         0.5
7  ETHUSDT  30m  fast    32     1     0         0.009734         0.0
8  ETHUSDT   1h  fast    15     0     0              NaN         NaN
9  ETHUSDT   4h  fast     4     0     0              NaN         NaN

In [12]:

from pathlib import Path
import pandas as pd, itertools
from chan.zhongshu_hierarchy import build_zhongshu_hierarchy

ROOT = Path.cwd().resolve()
if ROOT.name == "Note":
    ROOT = ROOT.parents[0]
CHAN = ROOT / "chan_data"
OUTDIR = CHAN / "zhongshu_hierarchy_fast"
OUTDIR.mkdir(parents=True, exist_ok=True)

symbols = ["BTCUSDT", "ETHUSDT"]
tfs = ["5m", "15m", "30m", "1h", "4h"]
whiches = ["fast"]

records = []

for sym, tf, which in itertools.product(symbols, tfs, whiches):
    p = CHAN / f"zhongshu_{which}" / sym / f"{tf}_zhongshu.csv"
    if not p.exists():
        print(f"[skip] {sym}-{tf}: The file does not exist.")
        continue
    l1 = pd.read_csv(p, parse_dates=["timestamp_start","timestamp_end"])
    if l1.empty:
        print(f"[skip] {sym}-{tf}: Empty file")
        continue

    print(f"[run] {sym}-{tf} building hierarchy ...")
    levels, pmap = build_zhongshu_hierarchy(
        l1, max_levels=3, mode="discovery",
        min_ratio=0.0005, strict_alt=False,
        allow_overlap_zs=True, enable_quality=True,
    )

    out_dir = OUTDIR / sym
    out_dir.mkdir(parents=True, exist_ok=True)

    for i, lv in enumerate(levels, start=1):
        if lv is None or lv.empty:
            continue
        lv.to_csv(out_dir / f"{tf}_L{i}.csv", index=False)
        records.append({
            "symbol": sym,
            "tf": tf,
            "which": which,
            "level": f"L{i}",
            "count": len(lv),
            "width_median": lv["zs_high"].sub(lv["zs_low"]).median() if "zs_high" in lv else None
        })
        print(f"  [saved] {sym}-{tf} {i}: n={len(lv)}")

print("[done] ")

df_sum = pd.DataFrame(records)
display(df_sum.sort_values(["symbol","tf","level"]))


[run] BTCUSDT-5m building hierarchy ...
  [saved] BTCUSDT-5m 1: n=18
[run] BTCUSDT-15m building hierarchy ...
  [saved] BTCUSDT-15m 1: n=22
  [saved] BTCUSDT-15m 2: n=2
[run] BTCUSDT-30m building hierarchy ...
  [saved] BTCUSDT-30m 1: n=20
  [saved] BTCUSDT-30m 2: n=1
[run] BTCUSDT-1h building hierarchy ...
  [saved] BTCUSDT-1h 1: n=16
[run] BTCUSDT-4h building hierarchy ...
  [saved] BTCUSDT-4h 1: n=5
[run] ETHUSDT-5m building hierarchy ...
  [saved] ETHUSDT-5m 1: n=73
  [saved] ETHUSDT-5m 2: n=3
[run] ETHUSDT-15m building hierarchy ...
  [saved] ETHUSDT-15m 1: n=50
  [saved] ETHUSDT-15m 2: n=3
[run] ETHUSDT-30m building hierarchy ...
  [saved] ETHUSDT-30m 1: n=32
  [saved] ETHUSDT-30m 2: n=1
[run] ETHUSDT-1h building hierarchy ...
  [saved] ETHUSDT-1h 1: n=15
[run] ETHUSDT-4h building hierarchy ...
  [saved] ETHUSDT-4h 1: n=4
[done] 所有 L1/L2/L3 已落盘。


symbol   tf which level  count  width_median
1   BTCUSDT  15m  fast    L1     22      1010.190
2   BTCUSDT  15m  fast    L2      2       745.395
5   BTCUSDT   1h  fast    L1     16      1174.895
3   BTCUSDT  30m  fast    L1     20      1121.425
4   BTCUSDT  30m  fast    L2      1      1264.250
6   BTCUSDT   4h  fast    L1      5      2172.820
0   BTCUSDT   5m  fast    L1     18      1101.085
9   ETHUSDT  15m  fast    L1     50        39.150
10  ETHUSDT  15m  fast    L2      3        10.020
13  ETHUSDT   1h  fast    L1     15        60.000
11  ETHUSDT  30m  fast    L1     32        50.105
12  ETHUSDT  30m  fast    L2      1        24.520
14  ETHUSDT   4h  fast    L1      4       160.885
7   ETHUSDT   5m  fast    L1     73        33.410
8   ETHUSDT   5m  fast    L2      3        15.280